In [42]:
import requests
import json

url = "http://localhost:11434/api/generate"
payload = {
    "model": "llama3.2:3b",
    "prompt": "너는 반드시 한국어로만 대답해야 한다. 대한민국의 수도는 어디야?"
}


with requests.post(url, json=payload, stream=True) as r:
    for line in r.iter_lines():
        if line:
            data = json.loads(line.decode("utf-8"))
            if "response" in data:
                print(data["response"], end="", flush=True)
            if data.get("done", False):
                break


한국의 수도인 서울입니다.

In [43]:
# 경로 설정
PDF_FILE_PATH = '/Users/yoodongseok/Desktop/rag_project/data/2024 관세행정 민원상담 사례집.pdf'

In [44]:
# 텍스트 추출
import os
import pdfplumber

all_text = ""
if PDF_FILE_PATH and os.path.exists(PDF_FILE_PATH): # 파일 존재 여부 확인
    with pdfplumber.open(PDF_FILE_PATH) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text: 
                all_text += f"--- 페이지 {i+1} ---\n{text}\n\n"
    
    print("--- 추출된 텍스트 미리보기 (상위 500자) ---")
    print(all_text[:500])
else:
    print("파일 경로가 올바르지 않거나 파일이 존재하지 않습니다. 경로를 다시 확인해 주세요.")

--- 추출된 텍스트 미리보기 (상위 500자) ---
--- 페이지 1 ---
발 간 등 록 번 호
11-1220000-000430-10
2024
관세행정 민원상담 사례집

--- 페이지 2 ---
일러두기
이 책은 그동안의 관세행정 민원상담 사례를 모아 민원인에게
도움을 주고자 최신법령 등을 반영한 종합민원 사례집으로, 관련
법령 개정 및 민원의 구체적인 사항에 따라 이 책의 내용이 적용될
수 없을 수 있습니다.
따라서, 이 책의 내용을 쟁송, 유권해석 등의 근거자료로 사용하는
것은 적절하지 아니하며, 전반적인 관세행정에 관한 참고자료로
활용해 주시고, 관련 법령 등을 확인하여 주시기 바랍니다.

--- 페이지 3 ---
발 간 사
관세청은, 관세행정 전 분야의 끊임없는
혁신을 통해 대한민국이 글로벌 중추국가로
도약할 수 있도록 앞장서 나가겠습니다.
최근 관세행정은 해외직구 등 전자상거래 관련 개인 민원이 2023년 전체 민원
총 240,103건에서 54%를 차지할 정도로 특송화물‧우편물‧여행자 분야의
중심 고객이 기업에서 개인으로 


In [45]:
# 테이블 추출
import tabula
import pandas as pd
import os

if PDF_FILE_PATH and os.path.exists(PDF_FILE_PATH): # 파일 존재 여부 확인
    print("\n[Tabula-py] 표 데이터 추출 시작...")
    
    dfs_list = tabula.read_pdf(
        PDF_FILE_PATH, 
        pages='all',
        multiple_tables=True,
        stream=True, 
        encoding='utf-8'
    )

    if dfs_list:
        print(f"총 {len(dfs_list)}개의 표(DataFrame)를 추출했습니다.")
        for i, df in enumerate(dfs_list):
            print(f"\n--- 추출된 표 {i+1} (DataFrame.head()) ---")
            print(df.head())
    else:
        print("Tabula-py가 PDF 파일에서 표를 찾지 못했습니다.")
else:
    print("파일 경로가 올바르지 않거나 파일이 존재하지 않습니다. 경로를 다시 확인해 주세요.")

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'



[Tabula-py] 표 데이터 추출 시작...


Got stderr: 10월 04, 2025 12:19:18 오후 org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
10월 04, 2025 12:19:18 오후 org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode



총 28개의 표(DataFrame)를 추출했습니다.

--- 추출된 표 1 (DataFrame.head()) ---
  Unnamed: 0   품목     물품가격        면세수량           비고
0        NaN   주류      NaN  1병 (1L 이하)  주세, 교육세는 과세
1        NaN   궐련      NaN       200개비          NaN
2        NaN  엽궐련      NaN        50개비          NaN
3        NaN  NaN  미화 150불         NaN       개별소비세,
4        NaN  NaN      NaN  니코틴용액 20ml          NaN

--- 추출된 표 2 (DataFrame.head()) ---
  과세가격결정방법 사전심사신청서【일반용】
0          (처리기간 : 1개월)
1     신  1 상 호  2 대 표 자
2                     청
3       인  3 주 소 (전화번호)
4            4 수입 예정 물품

--- 추출된 표 3 (DataFrame.head()) ---
  Unnamed: 0     과세가격 결정방법 사전심사
0        NaN        신청서【특수관계자용】
1        NaN        (처리기간 : 1년)
2          신  1 법인명(상호) 2 대 표 자
3          청                NaN
4          인       3 주 소 (전화번호)

--- 추출된 표 4 (DataFrame.head()) ---
    품목(예시) 면세통관범위 품목(예시).1 면세통관범위.1
0      참기름    5kg        잣      1kg
1      참 깨    5kg      쇠고기     10kg
2        꿀    5kg     돼지고기     10KG
3  고사리, 더덕    5kg      그 외  품목당 5kg

In [51]:
import pdfplumber
import pandas as pd

file_path = PDF_FILE_PATH
qa_pairs = []

with pdfplumber.open(file_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        tables = page.extract_tables()

        lines = text.split("\n") if text else []
        key = None
        value_lines = []

        for line in lines:
            if "?" in line:  # 질문
                if key and (value_lines or tables):
                    # 표를 문자열로 합쳐서 추가
                    table_text = ""
                    if tables:
                        for t in tables:
                            for row in t:
                                table_text += " | ".join([c if c else "" for c in row]) + "\n"

                    qa_pairs.append({
                        "question": key,
                        "answer": " ".join(value_lines).strip() + "\n" + table_text.strip()
                    })
                    value_lines = []
                key = line.strip()
            elif key:
                value_lines.append(line.strip())

        if key and (value_lines or tables):
            table_text = ""
            if tables:
                for t in tables:
                    for row in t:
                        table_text += " | ".join([c if c else "" for c in row]) + "\n"

            qa_pairs.append({
                "question": key,
                "answer": " ".join(value_lines).strip() + "\n" + table_text.strip()
            })

# DataFrame → Parquet 저장
df = pd.DataFrame(qa_pairs)
df.to_parquet("qa_dataset.parquet", engine="fastparquet", index=False)
print("✅ Parquet 저장 완료")
print(df.head())


✅ Parquet 저장 완료
                                            question  \
0  10. 세관장확인물품으로 지정된 유아용 섬유제품이란?·················...   
1  20. 전기용품 수입시 어떤 절차를 거쳐야 하나요?··················...   
2  24. 환불받은 해외직구물품 타인 명의로 다시 수입 가능한지?············...   
3  26. 수입 신고는 어느 세관에 하여야 하나?·····················...   
4  28. 개인자격으로 영리목적 수입 가능한지?······················...   

                                              answer  
0  11. 주방용 쓰레기 처리기(Disposers) 수입요건···············...  
1  21. 술, 담배의 면세기준과 기준 초과시 수입절차··················...  
2  25. 수입물품에 대한 세액계산 방법··························...  
3                                관세행정 민원상담 사례집 | i\n  
4  29. B/L 분할 수입신고 요건 및 B/L 합병 후 수입신고 요건·········...  


In [55]:
# 질문을 임베딩하여 Redis에 저장
# question을 임베딩해서 저장, answer는 그대로 저장
import redis
import numpy as np
import pandas as pd
import requests
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.index_definition import IndexDefinition, IndexType


# Redis 연결
r = redis.Redis(host="localhost", port=6379, decode_responses=False)

# Ollama 임베딩 함수
def get_embedding(text: str):
    url = "http://localhost:11434/api/embeddings"
    payload = {"model": "nomic-embed-text", "prompt": text}
    res = requests.post(url, json=payload, timeout=60)
    data = res.json()
    return data["embedding"]

# Parquet 로드
df = pd.read_parquet("qa_dataset.parquet", engine="fastparquet")

# 인덱스 생성
index_name = "qa_index"
try:
    r.ft(index_name).dropindex(delete_documents=True)
except:
    pass

dim = len(get_embedding("임베딩 차원 확인"))

r.ft(index_name).create_index(
    fields=[
        VectorField("embedding", "FLAT", {
            "TYPE": "FLOAT32",
            "DIM": dim,
            "DISTANCE_METRIC": "COSINE"
        }),
        TextField("answer")
    ],
    definition=IndexDefinition(prefix=["doc:"], index_type=IndexType.HASH)
)

# Redis에 적재
for i, row in df.iterrows():
    emb = np.array(get_embedding(row["question"]), dtype=np.float32).tobytes()
    r.hset(f"doc:{i}", mapping={
        "embedding": emb,
        "answer": row["answer"]
    })

print("✅ Redis에 임베딩 저장 완료")


✅ Redis에 임베딩 저장 완료


In [56]:
# 사용자 질의 → 임베딩 자동 생성
# LLM 입력 전에 사용자의 query를 임베딩
def embed_query(query: str):
    return np.array(get_embedding(query), dtype=np.float32).tobytes()

In [61]:
# Redis에서 유사 검색 → 답변 반환
# Query 임베딩과 저장된 embedding 비교 → 가장 유사한 answer 반환

from redis.commands.search.query import Query

def search_answer(query: str):
    q_emb = embed_query(query)

    q = Query('*=>[KNN 1 @embedding $vec]') \
        .return_fields("answer", "__score__") \
        .dialect(2)

    res = r.ft(index_name).search(q, query_params={"vec": q_emb})
    if res.docs:
        return res.docs[0]["answer"]
    else:
        return "❌ 답변을 찾을 수 없습니다."

# 테스트
query = "관세에 관해서 담배 규정이 뭐야?"
print("🔍 사용자 질의:", query)
print("📌 검색된 답변:", search_answer(query))

🔍 사용자 질의: 관세에 관해서 담배 규정이 뭐야?
📌 검색된 답변: 심 사 ‘과세가격 결정방법 사전심사’는 납세자가 수입신고를 할 때 세관장에게 납세신고를 하여야 하는데, 과세가격 결정과 관련하여 의문이 있을 경우 관세법 제37조에 따라 납세신고를 하기 전에 관세청장(관세평가분류원 장에게 위임)에게 미리 심사하여 줄 것을 신청할 수 있는 제도입니다. 이러한 과세가격 결정방법 사전심사에는 ‘일반 사전심사’와 ‘ACVA(특수 관계자간 과세가격 결정방법 사전심사)’가 있습니다. 일반 사전심사’는 특수관계가 없는 자들 간에 거래되는 수입물품에 대해 관세법 제30조제1항부터 제3항까지에 규정된 사항, 제30조에 따른 방 법으로 과세가격을 결정할 수 없는 경우에 적용되는 과세가격 결정방법에 대한 심사입니다. 반면 ‘ACVA’는 특수관계가 있는 자들 간에 거래되는 물품의 과세가격 결정방법에 대한 심사입니다. 일반 사전심사 대상은 성실납세 지원을 위해 2019년부터 관세법 제30조 뿐만 아니라 제31조부터 제35조까지를 포함한 과세가격 전반으로 확대 되었습니다. 관련법령 ● 관세법 제37조 PART 2 심 사|745
심
사
